# API Data Extractor
Last code chunk is what worked. I'll clean up this notebook this week and refine the data extraction process.

In [42]:
import requests
import json
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import time
import certifi
import random
from datetime import datetime, timedelta



# Pull articles with sentiment scores

In [ ]:
import certifi
import json
from urllib.request import urlopen
import time
import requests
from bs4 import BeautifulSoup

def get_jsonparsed_data(url):
    """
    Fetch JSON data from the given URL.

    Parameters:
    url (str): The URL to fetch data from.

    Returns:
    dict: The parsed JSON data.
    """
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)

def fetch_stock_news_sentiments(api_key, ticker, page=0):
    """
    Fetch stock news sentiments from Financial Modeling Prep API.

    Parameters:
    api_key (str): Your API key for Financial Modeling Prep.
    ticker (str): The stock ticker symbol (e.g., 'AAPL' for Apple).
    page (int): Page number for pagination. Default is 0.

    Returns:
    dict: The parsed JSON data containing news sentiments.
    """
    url = f"https://financialmodelingprep.com/api/v4/stock-news-sentiments-rss-feed?symbol={ticker}&page={page}&apikey={api_key}"
    return get_jsonparsed_data(url)

def save_news_sentiments_to_file(all_news_sentiments, filename):
    """
    Save all news sentiments to a single JSON file.

    Parameters:
    all_news_sentiments (list): The list of all news sentiments to save.
    filename (str): The name of the file to save the news sentiments.
    """
    with open(filename, 'w') as file:
        json.dump(all_news_sentiments, file, indent=4)
    print(f"All news sentiments saved to {filename}")

def fetch_and_save_all_news(api_key, tickers, page=0):
    """
    Fetch and save news articles for a list of tickers into a single JSON file.

    Parameters:
    api_key (str): Your API key for Financial Modeling Prep.
    tickers (list): A list of ticker symbols.
    page (int): Page number for pagination. Default is 0.
    """
    all_news_sentiments = []
    for ticker in tickers:
        print(f"Fetching news for {ticker}...")
        news_sentiments = fetch_stock_news_sentiments(api_key, ticker, page)
        if news_sentiments:
            all_news_sentiments.extend(news_sentiments)
        time.sleep(1)  # Sleep to avoid hitting API rate limits

    save_news_sentiments_to_file(all_news_sentiments, 'all_news_sentiments.json')

def fetch_sp500_tickers():
    """
    Fetch the list of S&P 500 ticker symbols from Wikipedia.

    Returns:
    list: A list of S&P 500 ticker symbols.
    """
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to fetch data from Wikipedia: {response.status_code}")
        return []

    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    tickers = []

    for row in table.find_all('tr')[1:]:
        ticker = row.find_all('td')[0].text.strip()
        tickers.append(ticker)

    return tickers



In [ ]:
# Example usage
api_key = '28a3eef526c43ab5888ab02222aada18'
# fetch top # tickers
#sp500_tickers = sp500_tickers[:5]  
sp500_tickers = fetch_sp500_tickers()

if sp500_tickers:
    fetch_and_save_all_news(api_key, sp500_tickers)

## Convert to pandas dataframe

In [31]:
import pandas as pd
import json

def json_to_dataframe(json_file):
    """
    Convert JSON file to a pandas DataFrame.

    Parameters:
    json_file (str): The path to the JSON file.

    Returns:
    DataFrame: A pandas DataFrame containing the JSON data.
    """
    with open(json_file, 'r') as file:
        data = json.load(file)
    
    # Create a DataFrame from the list of articles
    df = pd.DataFrame(data)
    return df

# Example usage
json_file = 'test.json'
df = json_to_dataframe(json_file)
print(df.head())  # Print the first few rows of the DataFrame

  symbol             publishedDate  \
0    NOK  2024-06-28T16:16:07.000Z   
1   MASI  2024-06-28T16:16:04.000Z   
2   AAPX  2024-06-28T16:16:01.000Z   
3    MFC  2024-06-28T16:15:59.000Z   
4   PATH  2024-06-28T16:15:00.000Z   

                                               title  \
0  Nokia (NOK) Set to Reboot Infrastructure Busin...   
1  Masimo (MASI), Cleveland Clinic Unite to Enhan...   
2  5 Leveraged ETFs That Have Gained in Double Di...   
3  Manulife Financial (MFC) Shares Gain on New Fi...   
4  Edelson Lechtzin LLP Urges UiPath, Inc. (NYSE:...   

                                               image            site  \
0  https://cdn.snapi.dev/images/v1/i/a/communicat...       zacks.com   
1  https://cdn.snapi.dev/images/v1/u/9/medical25-...       zacks.com   
2  https://cdn.snapi.dev/images/v1/t/n/etf30-2502...       zacks.com   
3  https://cdn.snapi.dev/images/v1/b/l/insurance9...       zacks.com   
4  https://cdn.snapi.dev/images/v1/g/w/press15-25...  accesswire.com   

 

### Write to CSV and Parquet

In [32]:
df.to_csv(f'{savename}.csv', index=False)

In [ ]:
df.to_parquet('all_news_sentiments.parquet', index=False)

# Pull Articles with sentiment scores, adjust date range

In [29]:
import certifi
import json
from urllib.request import urlopen
import time
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

def get_jsonparsed_data(url):
    """
    Fetch JSON data from the given URL.

    Parameters:
    url (str): The URL to fetch data from.

    Returns:
    dict: The parsed JSON data.
    """
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)

def fetch_stock_news_sentiments(api_key, ticker, date_from, date_to, limit=50):
    """
    Fetch stock news sentiments from Financial Modeling Prep API.

    Parameters:
    api_key (str): Your API key for Financial Modeling Prep.
    ticker (str): The stock ticker symbol (e.g., 'AAPL' for Apple).
    date_from (str): Start date for fetching articles (YYYY-MM-DD).
    date_to (str): End date for fetching articles (YYYY-MM-DD).
    limit (int): Number of articles to fetch per request. Default is 50.

    Returns:
    list: The parsed JSON data containing news sentiments.
    """
    url = f"https://financialmodelingprep.com/api/v4/stock-news-sentiments-rss-feed?symbol={ticker}&from={date_from}&to={date_to}&apikey={api_key}&limit={limit}"
    return get_jsonparsed_data(url)

def save_news_sentiments_to_file(all_news_sentiments, filename):
    """
    Save all news sentiments to a single JSON file.

    Parameters:
    all_news_sentiments (list): The list of all news sentiments to save.
    filename (str): The name of the file to save the news sentiments.
    """
    with open(filename, 'w') as file:
        json.dump(all_news_sentiments, file, indent=4)
    print(f"All news sentiments saved to {filename}")

def fetch_and_save_all_news(api_key, tickers, start_date, end_date, interval_days=7, limit=10):
    """
    Fetch and save news articles for a list of tickers into a single JSON file over a specified timeframe.

    Parameters:
    api_key (str): Your API key for Financial Modeling Prep.
    tickers (list): A list of ticker symbols.
    start_date (str): Start date for fetching articles (YYYY-MM-DD).
    end_date (str): End date for fetching articles (YYYY-MM-DD).
    interval_days (int): Number of days in each interval for fetching articles. Default is 7.
    limit (int): Number of articles to fetch per request. Default is 50.
    """
    all_news_sentiments = []
    current_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.strptime(end_date, '%Y-%m-%d')

    while current_date < end_date:
        next_date = current_date + timedelta(days=interval_days)
        for ticker in tickers:
            print(f"Fetching news for {ticker} from {current_date.strftime('%Y-%m-%d')} to {next_date.strftime('%Y-%m-%d')}...")
            news_sentiments = fetch_stock_news_sentiments(api_key, ticker, current_date.strftime('%Y-%m-%d'), next_date.strftime('%Y-%m-%d'), limit)
            if news_sentiments:
                all_news_sentiments.extend(news_sentiments)
            time.sleep(1)  # Sleep to avoid hitting API rate limits
        current_date = next_date

    save_news_sentiments_to_file(all_news_sentiments, f'{savename}.json')

def fetch_sp500_tickers():
    """
    Fetch the list of S&P 500 ticker symbols from Wikipedia.

    Returns:
    list: A list of S&P 500 ticker symbols.
    """
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to fetch data from Wikipedia: {response.status_code}")
        return []

    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    tickers = []

    for row in table.find_all('tr')[1:]:
        ticker = row.find_all('td')[0].text.strip()
        tickers.append(ticker)

    return tickers



In [35]:
# Example usage
api_key = '28a3eef526c43ab5888ab02222aada18'
sp500_tickers = fetch_sp500_tickers()
sp500_tickers = sp500_tickers[:1]  # Fetch news for the first n tickers
savename = 'test'

# Specify the date range
start_date = '2023-01-01'
end_date = '2023-02-28'

if sp500_tickers:
    fetch_and_save_all_news(api_key, ['AAPL'], start_date, end_date)

Fetching news for AAPL from 2023-01-01 to 2023-01-08...


/var/folders/j2/skjc081x2h5cd717r89c0q480000gn/T/ipykernel_80729/34511153.py:19: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


Fetching news for AAPL from 2023-01-08 to 2023-01-15...
Fetching news for AAPL from 2023-01-15 to 2023-01-22...
Fetching news for AAPL from 2023-01-22 to 2023-01-29...
Fetching news for AAPL from 2023-01-29 to 2023-02-05...
Fetching news for AAPL from 2023-02-05 to 2023-02-12...
Fetching news for AAPL from 2023-02-12 to 2023-02-19...
Fetching news for AAPL from 2023-02-19 to 2023-02-26...
Fetching news for AAPL from 2023-02-26 to 2023-03-05...
All news sentiments saved to test.json


## Convert to pandas dataframe

In [ ]:
import pandas as pd
import json

def json_to_dataframe(json_file):
    """
    Convert JSON file to a pandas DataFrame.

    Parameters:
    json_file (str): The path to the JSON file.

    Returns:
    DataFrame: A pandas DataFrame containing the JSON data.
    """
    with open(json_file, 'r') as file:
        data = json.load(file)
    
    # Create a DataFrame from the list of articles
    df = pd.DataFrame(data)
    return df

# Example usage
json_file = 'test.json'
df = json_to_dataframe(json_file)
print(df.head())  # Print the first few rows of the DataFrame

  symbol             publishedDate  \
0    NOK  2024-06-28T16:16:07.000Z   
1   MASI  2024-06-28T16:16:04.000Z   
2   AAPX  2024-06-28T16:16:01.000Z   
3    MFC  2024-06-28T16:15:59.000Z   
4   PATH  2024-06-28T16:15:00.000Z   

                                               title  \
0  Nokia (NOK) Set to Reboot Infrastructure Busin...   
1  Masimo (MASI), Cleveland Clinic Unite to Enhan...   
2  5 Leveraged ETFs That Have Gained in Double Di...   
3  Manulife Financial (MFC) Shares Gain on New Fi...   
4  Edelson Lechtzin LLP Urges UiPath, Inc. (NYSE:...   

                                               image            site  \
0  https://cdn.snapi.dev/images/v1/i/a/communicat...       zacks.com   
1  https://cdn.snapi.dev/images/v1/u/9/medical25-...       zacks.com   
2  https://cdn.snapi.dev/images/v1/t/n/etf30-2502...       zacks.com   
3  https://cdn.snapi.dev/images/v1/b/l/insurance9...       zacks.com   
4  https://cdn.snapi.dev/images/v1/g/w/press15-25...  accesswire.com   

 

### Write to CSV and Parquet

In [ ]:
df.to_csv(f'{savename}.csv', index=False)

In [ ]:
df.to_parquet('all_news_sentiments.parquet', index=False)

# General Stock news Extractor
For extracting general stock news. 

In [38]:

def fetch_stock_news(api_key, ticker, date_from, date_to, limit=50):
    """
    Fetch stock news from Financial Modeling Prep API.

    Parameters:
    api_key (str): Your API key for Financial Modeling Prep.
    ticker (str): The stock ticker symbol (e.g., 'AAPL' for Apple).
    date_from (str): Start date for fetching articles (YYYY-MM-DD).
    date_to (str): End date for fetching articles (YYYY-MM-DD).
    limit (int): Number of articles to fetch per request. Default is 50.

    Returns:
    list: The parsed JSON data containing news articles.
    """
    url = f"https://financialmodelingprep.com/api/v3/stock_news?tickers={ticker}&from={date_from}&to={date_to}&limit={limit}&apikey={api_key}"
    return get_jsonparsed_data(url)

def save_news_to_file(all_news, filename):
    """
    Save all news articles to a single JSON file.

    Parameters:
    all_news (list): The list of all news articles to save.
    filename (str): The name of the file to save the news articles.
    """
    with open(filename, 'w') as file:
        json.dump(all_news, file, indent=4)
    print(f"All news articles saved to {filename}")

def fetch_and_save_all_news(api_key, tickers, start_date, end_date, interval_days=7, max_articles_per_interval=50):
    """
    Fetch and save news articles for a list of tickers into a single JSON file over a specified timeframe.

    Parameters:
    api_key (str): Your API key for Financial Modeling Prep.
    tickers (list): A list of ticker symbols.
    start_date (str): Start date for fetching articles (YYYY-MM-DD).
    end_date (str): End date for fetching articles (YYYY-MM-DD).
    interval_days (int): Number of days in each interval for fetching articles. Default is 7.
    max_articles_per_interval (int): Maximum number of articles to fetch per interval. Default is 50.
    """
    all_news = []
    current_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.strptime(end_date, '%Y-%m-%d')

    while current_date < end_date:
        next_date = current_date + timedelta(days=interval_days)
        for ticker in tickers:
            print(f"Fetching news for {ticker} from {current_date.strftime('%Y-%m-%d')} to {next_date.strftime('%Y-%m-%d')}...")
            random_start_date = current_date + timedelta(days=random.randint(0, interval_days - 1))
            random_end_date = random_start_date + timedelta(days=random.randint(1, interval_days - (random_start_date - current_date).days))
            limit = random.randint(1, max_articles_per_interval)
            news_articles = fetch_stock_news(api_key, ticker, random_start_date.strftime('%Y-%m-%d'), random_end_date.strftime('%Y-%m-%d'), limit)
            if news_articles:
                # Filter articles to include only those for the specified ticker
                filtered_news_articles = [article for article in news_articles if ticker in article.get('symbol', '')]
                all_news.extend(filtered_news_articles)
            time.sleep(1)  # Sleep to avoid hitting API rate limits
        current_date = next_date

    save_news_to_file(all_news, 'all_stock_news.json')




In [41]:
# Example usage
api_key = '28a3eef526c43ab5888ab02222aada18'
sp500_tickers = fetch_sp500_tickers()

# Limit the tickers for testing
limited_tickers = sp500_tickers[:1]

# Specify the date range
start_date = '2023-01-01'
end_date = '2023-02-05'

if limited_tickers:
    fetch_and_save_all_news(api_key, limited_tickers, start_date, end_date)

Fetching news for MMM from 2023-01-01 to 2023-01-08...


/var/folders/j2/skjc081x2h5cd717r89c0q480000gn/T/ipykernel_80729/34511153.py:19: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


Fetching news for MMM from 2023-01-08 to 2023-01-15...
Fetching news for MMM from 2023-01-15 to 2023-01-22...
Fetching news for MMM from 2023-01-22 to 2023-01-29...
Fetching news for MMM from 2023-01-29 to 2023-02-05...
All news articles saved to all_stock_news.json


## Convert to pandas dataframe

In [ ]:
import pandas as pd
import json

def json_to_dataframe(json_file):
    """
    Convert JSON file to a pandas DataFrame.

    Parameters:
    json_file (str): The path to the JSON file.

    Returns:
    DataFrame: A pandas DataFrame containing the JSON data.
    """
    with open(json_file, 'r') as file:
        data = json.load(file)
    
    # Create a DataFrame from the list of articles
    df = pd.DataFrame(data)
    return df

# Example usage
json_file = 'test.json'
df = json_to_dataframe(json_file)
print(df.head())  # Print the first few rows of the DataFrame

  symbol             publishedDate  \
0    NOK  2024-06-28T16:16:07.000Z   
1   MASI  2024-06-28T16:16:04.000Z   
2   AAPX  2024-06-28T16:16:01.000Z   
3    MFC  2024-06-28T16:15:59.000Z   
4   PATH  2024-06-28T16:15:00.000Z   

                                               title  \
0  Nokia (NOK) Set to Reboot Infrastructure Busin...   
1  Masimo (MASI), Cleveland Clinic Unite to Enhan...   
2  5 Leveraged ETFs That Have Gained in Double Di...   
3  Manulife Financial (MFC) Shares Gain on New Fi...   
4  Edelson Lechtzin LLP Urges UiPath, Inc. (NYSE:...   

                                               image            site  \
0  https://cdn.snapi.dev/images/v1/i/a/communicat...       zacks.com   
1  https://cdn.snapi.dev/images/v1/u/9/medical25-...       zacks.com   
2  https://cdn.snapi.dev/images/v1/t/n/etf30-2502...       zacks.com   
3  https://cdn.snapi.dev/images/v1/b/l/insurance9...       zacks.com   
4  https://cdn.snapi.dev/images/v1/g/w/press15-25...  accesswire.com   

 

### Write to CSV and Parquet

In [ ]:
df.to_csv('all_news.csv', index=False)

In [ ]:
df.to_parquet('all_news.parquet', index=False)

# Pull Stock Price data

In [43]:

def fetch_historical_stock_price(api_key, ticker, date_from, date_to):
    """
    Fetch historical stock price data from Financial Modeling Prep API.

    Parameters:
    api_key (str): Your API key for Financial Modeling Prep.
    ticker (str): The stock ticker symbol (e.g., 'AAPL' for Apple).
    date_from (str): Start date for fetching data (YYYY-MM-DD).
    date_to (str): End date for fetching data (YYYY-MM-DD).

    Returns:
    list: The parsed JSON data containing historical stock prices.
    """
    url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{ticker}?from={date_from}&to={date_to}&apikey={api_key}"
    return get_jsonparsed_data(url)

def save_historical_prices_to_file(all_prices, filename):
    """
    Save all historical stock prices to a single JSON file.

    Parameters:
    all_prices (list): The list of all historical stock prices to save.
    filename (str): The name of the file to save the historical stock prices.
    """
    with open(filename, 'w') as file:
        json.dump(all_prices, file, indent=4)
    print(f"All historical stock prices saved to {filename}")

def fetch_and_save_all_historical_prices(api_key, tickers, start_date, end_date):
    """
    Fetch and save historical stock prices for a list of tickers into a single JSON file over a specified timeframe.

    Parameters:
    api_key (str): Your API key for Financial Modeling Prep.
    tickers (list): A list of ticker symbols.
    start_date (str): Start date for fetching data (YYYY-MM-DD).
    end_date (str): End date for fetching data (YYYY-MM-DD).
    """
    all_historical_prices = []
    for ticker in tickers:
        print(f"Fetching historical prices for {ticker} from {start_date} to {end_date}...")
        historical_prices = fetch_historical_stock_price(api_key, ticker, start_date, end_date)
        if historical_prices:
            all_historical_prices.append({ticker: historical_prices})
        time.sleep(1)  # Sleep to avoid hitting API rate limits

    save_historical_prices_to_file(all_historical_prices, 'all_historical_prices.json')

def fetch_sp500_tickers():
    """
    Fetch the list of S&P 500 ticker symbols from Wikipedia.

    Returns:
    list: A list of S&P 500 ticker symbols.
    """
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to fetch data from Wikipedia: {response.status_code}")
        return []

    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    tickers = []

    for row in table.find_all('tr')[1:]:
        ticker = row.find_all('td')[0].text.strip()
        tickers.append(ticker)

    return tickers



In [44]:
# Example usage
api_key = '28a3eef526c43ab5888ab02222aada18'
sp500_tickers = fetch_sp500_tickers()

# Limit to the first ticker (MMM)
limited_tickers = sp500_tickers #[:1]

# Specify the date range
start_date = '2023-01-01'
end_date = '2023-12-31'

if limited_tickers:
    fetch_and_save_all_historical_prices(api_key, limited_tickers, start_date, end_date)

Fetching historical prices for MMM from 2023-01-01 to 2023-12-31...


/var/folders/j2/skjc081x2h5cd717r89c0q480000gn/T/ipykernel_80729/34511153.py:19: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


Fetching historical prices for AOS from 2023-01-01 to 2023-12-31...
Fetching historical prices for ABT from 2023-01-01 to 2023-12-31...
Fetching historical prices for ABBV from 2023-01-01 to 2023-12-31...
Fetching historical prices for ACN from 2023-01-01 to 2023-12-31...
Fetching historical prices for ADBE from 2023-01-01 to 2023-12-31...
Fetching historical prices for AMD from 2023-01-01 to 2023-12-31...
Fetching historical prices for AES from 2023-01-01 to 2023-12-31...
Fetching historical prices for AFL from 2023-01-01 to 2023-12-31...
Fetching historical prices for A from 2023-01-01 to 2023-12-31...
Fetching historical prices for APD from 2023-01-01 to 2023-12-31...
Fetching historical prices for ABNB from 2023-01-01 to 2023-12-31...
Fetching historical prices for AKAM from 2023-01-01 to 2023-12-31...
Fetching historical prices for ALB from 2023-01-01 to 2023-12-31...
Fetching historical prices for ARE from 2023-01-01 to 2023-12-31...
Fetching historical prices for ALGN from 2023-

## Convert prices json data to pandas dataframe, csv, parquet

In [49]:
import json
import pandas as pd

def json_to_dataframe(json_file):
    """
    Convert JSON file to a pandas DataFrame in tidy data format.

    Parameters:
    json_file (str): The path to the JSON file.

    Returns:
    DataFrame: A pandas DataFrame in tidy data format.
    """
    with open(json_file, 'r') as file:
        data = json.load(file)
    
    records = []
    for ticker_data in data:
        for ticker, content in ticker_data.items():
            historical_data = content["historical"]
            for price in historical_data:
                price['ticker'] = ticker
                records.append(price)
    
    df = pd.DataFrame(records)
    return df

def save_dataframe_to_csv(df, csv_file):
    """
    Save DataFrame to a CSV file.

    Parameters:
    df (DataFrame): The pandas DataFrame to save.
    csv_file (str): The name of the CSV file.
    """
    df.to_csv(csv_file, index=False)
    print(f"Data saved to {csv_file}")

def save_dataframe_to_parquet(df, parquet_file):
    """
    Save DataFrame to a Parquet file.

    Parameters:
    df (DataFrame): The pandas DataFrame to save.
    parquet_file (str): The name of the Parquet file.
    """
    df.to_parquet(parquet_file, index=False)
    print(f"Data saved to {parquet_file}")

# Example usage
json_file = 'all_historical_prices.json'
csv_file = 'all_historical_prices.csv'
parquet_file = 'all_historical_prices.parquet'

# Convert JSON to DataFrame
df = json_to_dataframe(json_file)

# Check if all columns are retained
print("Columns in DataFrame:", df.columns)

# Save DataFrame to CSV
save_dataframe_to_csv(df, csv_file)

# Save DataFrame to Parquet
save_dataframe_to_parquet(df, parquet_file)

Columns in DataFrame: Index(['date', 'open', 'high', 'low', 'close', 'adjClose', 'volume',
       'unadjustedVolume', 'change', 'changePercent', 'vwap', 'label',
       'changeOverTime', 'ticker'],
      dtype='object')
Data saved to all_historical_prices.csv
Data saved to all_historical_prices.parquet
